# **AIM460 Capstone — Cross-Domain Algorithmic Fairness Benchmark**
**Runtime:** Google Colab (GPU recommended)

**Methods:** DSAP (pre), Equalized Odds via Exponentiated Gradient (in), Threshold Optimizer (post), FRAME/SLSD stubs, DeAR (vision).  
**Datasets:** ACSIncome, ACSEmployment, COMPAS, CivilComments (manual upload), FairFace (optional demo).
**Metrics:** Accuracy, AUC, EO gap (TPR), Eq-Odds gaps (TPR/TNR).
**Exports:** figures/CSVs in `/content/exports/` + release + Overleaf pack.

In [1]:
# 🚫 Disable W&B globally (prevents login prompt)
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"
os.environ["DISABLE_WANDB"] = "true"
print("✅ Global W&B disabled")

✅ Global W&B disabled


In [2]:
# 🧹 Remove wandb completely so nothing can call it
!pip -q uninstall -y wandb pathtools setproctitle docker-pycreds GitPython sentry-sdk

## 2) Colab Setup (Install)

## 2) Data Loading


In [3]:
# PURPOSE: Load datasets and align schemas
# - Read ACSIncome, ACSEmployment (Folktables), COMPAS, CivilComments
# - Select features/target; cast types; attach sensitive attributes for EVAL ONLY


#@title Install Runtime Dependencies (CUDA 12.1 stack)
!pip -q install --upgrade pip
!pip -q install "pandas==2.2.2" "scikit-learn>=1.4,<1.6" "matplotlib>=3.8,<3.10" "shap>=0.45,<0.47"
!pip -q install "fairlearn>=0.12,<0.14" "folktables>=0.0.12" "datasets>=2.20,<2.22" "evaluate>=0.4,<0.5" "transformers>=4.44,<4.47"
!pip -q install --index-url https://download.pytorch.org/whl/cu121 "torch==2.4.0" "torchaudio==2.4.0" "torchvision==0.19.0"
!pip -q install "nbformat>=5.10,<6" "python-pptx>=0.6.21,<0.7" "tabulate>=0.9,<0.10"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.


In [4]:
#@title Conflict Clean-up (safe to run once after installing dependencies)
# Removes unused packages that force incompatible pins in Colab.
# Run this *after* the Install cell finishes, then go to: Runtime → Restart runtime.
!pip -q uninstall -y umap-learn gcsfs
!pip -q install "scikit-learn==1.5.2" "fsspec==2024.6.1" --no-deps

After installs: **Runtime → Restart runtime**, then run **Environment Check**.

In [1]:
# PURPOSE: Load datasets and align schemas
# - Read ACSIncome, ACSEmployment (Folktables), COMPAS, CivilComments
# - Select features/target; cast types; attach sensitive attributes for EVAL ONLY


#@title Environment Check
import importlib, torch

def ok(v): return f"✅ {v}"
def warn(v): return f"⚠️ {v}"

def check_version(pkg, pref=None):
    try:
        m = importlib.import_module(pkg); v = getattr(m, "__version__", "unknown")
        return ok(f"{pkg}=={v}") if (pref is None or v.startswith(pref)) else warn(f"{pkg}=={v} (expect {pref})")
    except Exception as e:
        return warn(f"{pkg} import failed: {e}")

print(check_version("torch","2.4"))
print(ok(f"CUDA available: {torch.cuda.is_available()}"))
if torch.cuda.is_available():
    print(ok(f"Device: {torch.cuda.get_device_name(0)}")); print(ok(f"CUDA: {torch.version.cuda}"))
for p, pref in [("sklearn",None),("pandas","2.2"),("matplotlib",None),("fairlearn",None),("folktables",None),("datasets",None),("evaluate",None),("transformers",None),("shap",None)]:
    print(check_version(p,pref))

✅ torch==2.4.0+cu121
✅ CUDA available: True
✅ Device: Tesla T4
✅ CUDA: 12.1
✅ sklearn==1.5.2
✅ pandas==2.2.2
✅ matplotlib==3.9.4
✅ fairlearn==0.13.0
✅ folktables==0.0.12
✅ datasets==2.21.0
✅ evaluate==0.4.6
✅ transformers==4.46.3
✅ shap==0.46.0


### ✅ What to run next (after Environment Check)
1. Run **Utility Functions (metrics, plotting, exports)**  
2. **Folktables** → run **ACSIncome** then **ACSEmployment** (baseline → DSAP → Eq-Odds → Threshold Optimizer → plot)  
3. **COMPAS** → download/preprocess → methods → plot  
4. **CivilComments (Text)** → upload `train.csv` → move & verify → run baseline + plot  
5. **FairFace (Optional)** → run toy DeAR demo + plot  
6. **Aggregated Results** → write/preview `results_summary_all.csv`  
7. **Sanitize & Export** → creates cleaned notebook, release ZIP, and Overleaf pack

## 3) Utility Functions (metrics, plotting, exports)

## 1) Environment & Imports


In [2]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output

#@title Helper: Exponentiated Gradient Probability Extraction
import numpy as np

def eg_probabilities(eg, X):
    """Return class-1 probabilities for ExponentiatedGradient across fairlearn versions."""
    if hasattr(eg, "_pmf_predict"):
        try:
            pmf = eg._pmf_predict(X)
            return pmf[:, 1]
        except Exception:
            pass
    if hasattr(eg, "predictors_"):
        probs = []
        for est in eg.predictors_:
            if hasattr(est, "predict_proba"):
                probs.append(est.predict_proba(X)[:, 1])
            elif hasattr(est, "decision_function"):
                s = est.decision_function(X)
                probs.append(1 / (1 + np.exp(-s)))
            else:
                probs.append(est.predict(X).astype(float))
        P = np.vstack(probs)
        if hasattr(eg, "weights_") and eg.weights_ is not None:
            w = np.asarray(eg.weights_, dtype=float)
            w = w / w.sum()
            return (w @ P).ravel()
        else:
            return P.mean(axis=0)
    return eg.predict(X).astype(float)

In [3]:
#@title Helper: Threshold Optimizer Evaluation
def evaluate_threshold_optimizer(cal, to, X_test, y_test, g_test, dataset_name):
    """Evaluate calibrated probabilities + post-processed labels, then log summary."""
    y_score = cal.predict_proba(X_test)[:, 1]
    y_pred  = to.predict(X_test, sensitive_features=g_test)
    m_to = evaluate_binary(y_test, y_score, y_pred, g_test)
    add_summary_row(dataset_name, "threshold_optimizer", m_to)
    return m_to

In [4]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output

#@title Helper: Trade-off Plot + CSV Export (utility ↑ vs EO gap ↓)
import pandas as pd
def export_tradeoff_points(points, dataset_key, title):
    df = pd.DataFrame(points, columns=["label","accuracy","eo_tpr_gap"]).dropna()
    csv_path = f"/content/exports/{dataset_key}_points.csv"
    png_path = f"/content/exports/{dataset_key}_tradeoff.png"
    save_results_csv(df, csv_path)
    tradeoff_plot(df, "accuracy", "eo_tpr_gap", title, png_path)
    flush_summary_csv("/content/exports/results_summary_all.csv")
    return csv_path, png_path

In [5]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output


#@title Utilities
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score

EXPORT_DIR="/content/exports"; os.makedirs(EXPORT_DIR, exist_ok=True)

def tpr_fpr(y_true, y_pred, pos=1):
    y_true=np.asarray(y_true); y_pred=np.asarray(y_pred)
    tp=np.sum((y_true==pos)&(y_pred==pos)); fn=np.sum((y_true==pos)&(y_pred!=pos))
    tn=np.sum((y_true!=pos)&(y_pred!=pos)); fp=np.sum((y_true!=pos)&(y_pred==pos))
    tpr=tp/(tp+fn+1e-12); fpr=fp/(fp+tn+1e-12); tnr=1-fpr; return tpr,fpr,tnr

def group_gaps(y_true, y_pred, g):
    import pandas as pd, numpy as np
    df=pd.DataFrame({"y":y_true,"yp":y_pred,"g":g})
    tprs, tnrs = [], []
    for _,sub in df.groupby("g"):
        tpr, fpr, tnr = tpr_fpr(sub.y, sub.yp); tprs.append(tpr); tnrs.append(tnr)
    eo = float(np.max(tprs)-np.min(tprs)) if tprs else np.nan
    tnr_gap = float(np.max(tnrs)-np.min(tnrs)) if tnrs else np.nan
    return {"eo_tpr_gap":eo, "eqodds_tpr_gap":eo, "eqodds_tnr_gap":tnr_gap}

def evaluate_binary(y_true, y_score, y_pred, g):
    out={"accuracy": float(accuracy_score(y_true, y_pred))}
    try: out["auc"]=float(roc_auc_score(y_true, y_score))
    except: out["auc"]=float("nan")
    out.update(group_gaps(y_true, y_pred, g)); return out

def tradeoff_plot(df, x, y, title, out_png):
    plt.figure(); plt.scatter(df[x], df[y])
    for _,r in df.iterrows():
        if isinstance(r.get("label"), str): plt.annotate(r["label"], (r[x], r[y]))
    plt.xlabel(f"{x} (↑)"); plt.ylabel(f"{y} (↓)"); plt.title(title); plt.grid(True)
    plt.savefig(out_png, bbox_inches="tight"); plt.close()

def save_results_csv(df, path): df.to_csv(path, index=False)

RESULTS_SUMMARY=[]
def add_summary_row(dataset, method, metrics):
    row={"dataset":dataset,"method":method}; row.update(metrics); RESULTS_SUMMARY.append(row)
def flush_summary_csv(path="/content/exports/results_summary_all.csv"):
    pd.DataFrame(RESULTS_SUMMARY).to_csv(path, index=False) if RESULTS_SUMMARY else pd.DataFrame([],columns=["dataset","method","accuracy","auc","eo_tpr_gap","eqodds_tpr_gap","eqodds_tnr_gap"]).to_csv(path,index=False)

## 4) Folktables — ACSIncome

In [6]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output

#@title Aligned loaders for Folktables (use task-returned group)
from folktables import ACSDataSource, ACSIncome, ACSEmployment
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

def _binarize_group(g_task: np.ndarray) -> np.ndarray:
    """
    Convert Folktables' returned group vector to {0,1}.
    Commonly '1' corresponds to White in RAC1P; we map (==1) -> 1 else 0.
    If it's already 0/1, we return as-is.
    """
    g = np.asarray(g_task)
    if set(np.unique(g)).issubset({0, 1}):
        return g.astype(int)
    try:
        return (g.astype(int) == 1).astype(int)
    except Exception:
        return (g > np.median(g)).astype(int)

def load_acs_income(state="CA", year=2018, horizon="1-Year"):
    ds = ACSDataSource(survey_year=year, horizon=horizon, survey="person")
    acs = ds.get_data(download=True, states=[state])
    X, y, g_task = ACSIncome.df_to_numpy(acs)  # aligned group from task
    g = _binarize_group(g_task)
    return X, y, g

def load_acs_employment(state="CA", year=2018, horizon="1-Year"):
    ds = ACSDataSource(survey_year=year, horizon=horizon, survey="person")
    acs = ds.get_data(download=True, states=[state])
    X, y, g_task = ACSEmployment.df_to_numpy(acs)
    g = _binarize_group(g_task)
    return X, y, g

def split_scale(X, y, g, test_size=0.25, seed=42):
    Xtr, Xte, ytr, yte, gtr, gte = train_test_split(X, y, g, test_size=test_size, random_state=seed, stratify=y)
    sc = StandardScaler(with_mean=False)
    Xtr = sc.fit_transform(Xtr)
    Xte = sc.transform(Xte)
    return Xtr, Xte, ytr, yte, gtr, gte, sc

# --- Run income loader and baseline ---
X, y, g = load_acs_income()
Xtr, Xte, ytr, yte, gtr, gte, _ = split_scale(X, y, g)

base = LogisticRegression(max_iter=200, n_jobs=-1).fit(Xtr, ytr)
ys = base.predict_proba(Xte)[:, 1]
yp = (ys >= 0.5).astype(int)

m_base = evaluate_binary(yte, ys, yp, gte)
add_summary_row("acs_income", "baseline", m_base)
m_base


{'accuracy': 0.7845738700247358,
 'auc': 0.859781744336534,
 'eo_tpr_gap': 0.09241830535443785,
 'eqodds_tpr_gap': 0.09241830535443785,
 'eqodds_tnr_gap': 0.041503931805421335}

In [7]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output


#@title DSAP-style reweighting
from collections import Counter, defaultdict
import numpy as np
def dsap_reweight(y,g):
    pairs=list(zip(y,g)); c=Counter(pairs)
    w=np.array([1.0/c[(yy,gg)] for yy,gg in pairs],dtype=float); return w*(len(w)/w.sum())
w=dsap_reweight(ytr,gtr)
dsap=LogisticRegression(max_iter=200,n_jobs=-1).fit(Xtr,ytr,sample_weight=w)
ys2=dsap.predict_proba(Xte)[:,1]; yp2=(ys2>=0.5).astype(int)
m_dsap=evaluate_binary(yte, ys2, yp2, gte); add_summary_row("acs_income","dsap_reweight",m_dsap); m_dsap

{'accuracy': 0.7785841323057424,
 'auc': 0.8587561624177105,
 'eo_tpr_gap': 0.018239469510906492,
 'eqodds_tpr_gap': 0.018239469510906492,
 'eqodds_tnr_gap': 0.00031267896991249433}

In [8]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output


#@title Equalized Odds via ExponentiatedGradient
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds
from sklearn.tree import DecisionTreeClassifier
import numpy as np
eg=ExponentiatedGradient(DecisionTreeClassifier(max_depth=4,random_state=0), EqualizedOdds(), eps=0.02, max_iter=50)
eg.fit(Xtr,ytr,sensitive_features=gtr)
ys3 = eg_probabilities(eg, Xte)
yp3=(ys3>=0.5).astype(int)
m_eg=evaluate_binary(yte, ys3, yp3, gte); add_summary_row("acs_income","eq_odds_eg",m_eg); m_eg

{'accuracy': 0.7735552057566899,
 'auc': 0.7843569985977736,
 'eo_tpr_gap': 0.008965899666668498,
 'eqodds_tpr_gap': 0.008965899666668498,
 'eqodds_tnr_gap': 0.009792358677063184}

## 5) Baseline Models


In [9]:
# PURPOSE: Train baseline models
# - Tabular: LogisticRegression / tree baseline
# - Text: DistilBERT fine-tuning for CivilComments
# - Save validation scores for post-processing


#@title Threshold Optimizer (Post-Processing on Calibrated Scores)
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from fairlearn.postprocessing import ThresholdOptimizer
from sklearn.model_selection import train_test_split

# Split for calibration and validation
Xc_tr, Xc_val, yc_tr, yc_val, gc_tr, gc_val = train_test_split(
    Xtr, ytr, gtr, test_size=0.2, random_state=3, stratify=ytr
)

# 1️⃣ Fit the base logistic regression first
base_lr = LogisticRegression(max_iter=200, n_jobs=-1)
base_lr.fit(Xc_tr, yc_tr)

# 2️⃣ Calibrate probabilities (no prefit flag → CalibratedClassifierCV will refit internally)
cal = CalibratedClassifierCV(base_lr, cv=3, method="isotonic")
cal.fit(Xc_tr, yc_tr)

# 3️⃣ Fit Fairlearn ThresholdOptimizer for Equalized Odds
to = ThresholdOptimizer(
    estimator=cal,
    constraints="equalized_odds",
    predict_method="predict_proba"
)
to.fit(Xc_val, yc_val, sensitive_features=gc_val)

# 4️⃣ Evaluate calibrated + post-processed results
m_to = evaluate_threshold_optimizer(cal, to, Xte, yte, gte, "acs_income")
m_to

{'accuracy': 0.7793814011488849,
 'auc': 0.8598443934980591,
 'eo_tpr_gap': 0.013762846228842718,
 'eqodds_tpr_gap': 0.013762846228842718,
 'eqodds_tnr_gap': 0.00015310303195670283}

In [10]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output


#@title Plot & export (income)
import pandas as pd
df=pd.DataFrame([
    {"label":"baseline","accuracy":m_base["accuracy"],"eo_tpr_gap":m_base["eo_tpr_gap"]},
    {"label":"dsap","accuracy":m_dsap["accuracy"],"eo_tpr_gap":m_dsap["eo_tpr_gap"]},
    {"label":"eq_odds","accuracy":m_eg["accuracy"],"eo_tpr_gap":m_eg["eo_tpr_gap"]},
    {"label":"thresh_opt","accuracy":m_to["accuracy"],"eo_tpr_gap":m_to["eo_tpr_gap"]},
])
png="/content/exports/income_tradeoff.png"
tradeoff_plot(df,"accuracy","eo_tpr_gap","ACSIncome: Utility vs Fairness (EO gap)",png)
save_results_csv(df,"/content/exports/acs_income_points.csv"); flush_summary_csv(); png

'/content/exports/income_tradeoff.png'

## 5) Folktables — ACSEmployment

In [11]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output

#@title Folktables: ACSEmployment (Tabular Fairness Benchmark)
from folktables import ACSEmployment
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

# 1️⃣ Load and split
Xe, ye, ge = load_acs_employment()
XtrE, XteE, ytrE, yteE, gtrE, gteE, scE = split_scale(Xe, ye, ge)

# 2️⃣ Baseline Logistic Regression
baseE = LogisticRegression(max_iter=200, n_jobs=-1).fit(XtrE, ytrE)
ysEb = baseE.predict_proba(XteE)[:, 1]
ypEb = (ysEb >= 0.5).astype(int)
mEb = evaluate_binary(yteE, ysEb, ypEb, gteE)
add_summary_row("acs_employment", "baseline", mEb)
print("✅ Baseline metrics:")
display(mEb)

# 3️⃣ DSAP-style reweighting (pre-processing fairness)
from sklearn.utils.class_weight import compute_sample_weight
wE = compute_sample_weight("balanced", ytrE)
lr_dsapE = LogisticRegression(max_iter=200, n_jobs=-1)
lr_dsapE.fit(XtrE, ytrE, sample_weight=wE)
ysEd = lr_dsapE.predict_proba(XteE)[:, 1]
ypEd = (ysEd >= 0.5).astype(int)
mEd = evaluate_binary(yteE, ysEd, ypEd, gteE)
add_summary_row("acs_employment", "dsap", mEd)
print("✅ DSAP metrics:")
display(mEd)

# 4️⃣ Equalized Odds (ExponentiatedGradient)
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds
from sklearn.tree import DecisionTreeClassifier

egE = ExponentiatedGradient(
    DecisionTreeClassifier(max_depth=4, random_state=0),
    EqualizedOdds(),
    eps=0.02
)
egE.fit(XtrE, ytrE, sensitive_features=gtrE)

ysEe = eg_probabilities(egE, XteE)
ypEe = (ysEe >= 0.5).astype(int)
mEe = evaluate_binary(yteE, ysEe, ypEe, gteE)
add_summary_row("acs_employment", "eq_odds_eg", mEe)
print("✅ Equalized Odds (EG) metrics:")
display(mEe)

# 5️⃣ Threshold Optimizer (Post-Processing on Calibrated Scores)
from sklearn.calibration import CalibratedClassifierCV
from fairlearn.postprocessing import ThresholdOptimizer

# Split training data into calibration (train) and validation subsets
Xc_trE, Xc_valE, yc_trE, yc_valE, gc_trE, gc_valE = train_test_split(
    XtrE, ytrE, gtrE, test_size=0.2, random_state=3, stratify=ytrE
)

# Base classifier
base_lrE = LogisticRegression(max_iter=200, n_jobs=-1)
base_lrE.fit(Xc_trE, yc_trE)

# Calibrate probabilities (cv=3 avoids prefit issues)
calE = CalibratedClassifierCV(base_lrE, cv=3, method="isotonic")
calE.fit(Xc_trE, yc_trE)

# Threshold Optimizer for Equalized Odds
toE = ThresholdOptimizer(
    estimator=calE,
    constraints="equalized_odds",
    predict_method="predict_proba"
)
toE.fit(Xc_valE, yc_valE, sensitive_features=gc_valE)

# Evaluate calibrated + post-processed results
mE_to = evaluate_threshold_optimizer(calE, toE, XteE, yteE, gteE, "acs_employment")
print("✅ Threshold Optimizer metrics:")
display(mE_to)

# 6️⃣ Trade-off plot + CSV export
_ = export_tradeoff_points(
    [
        {"label":"baseline",   "accuracy": mEb["accuracy"], "eo_tpr_gap": mEb["eo_tpr_gap"]},
        {"label":"dsap",       "accuracy": mEd["accuracy"], "eo_tpr_gap": mEd["eo_tpr_gap"]},
        {"label":"eq_odds",    "accuracy": mEe["accuracy"], "eo_tpr_gap": mEe["eo_tpr_gap"]},
        {"label":"thresh_opt", "accuracy": mE_to["accuracy"], "eo_tpr_gap": mE_to["eo_tpr_gap"]},
    ],
    dataset_key="employment",
    title="ACSEmployment: Utility vs Fairness (EO gap)"
)

print("✅ Exported trade-off plot and CSV for ACSEmployment")

✅ Baseline metrics:


{'accuracy': 0.7716065677630537,
 'auc': 0.8452249818660857,
 'eo_tpr_gap': 0.017761207445413985,
 'eqodds_tpr_gap': 0.017761207445413985,
 'eqodds_tnr_gap': 0.02677564559123713}

✅ DSAP metrics:


{'accuracy': 0.7679953539939813,
 'auc': 0.8449961915562069,
 'eo_tpr_gap': 0.013635294558256295,
 'eqodds_tpr_gap': 0.013635294558256295,
 'eqodds_tnr_gap': 0.021149626801078902}

✅ Equalized Odds (EG) metrics:


{'accuracy': 0.8087957341217464,
 'auc': 0.8359837726968038,
 'eo_tpr_gap': 0.043088198943859846,
 'eqodds_tpr_gap': 0.043088198943859846,
 'eqodds_tnr_gap': 0.030995702234827727}

✅ Threshold Optimizer metrics:


{'accuracy': 0.7712369991024761,
 'auc': 0.8452943015583347,
 'eo_tpr_gap': 0.004291808387610119,
 'eqodds_tpr_gap': 0.004291808387610119,
 'eqodds_tnr_gap': 0.009960499750997664}

✅ Exported trade-off plot and CSV for ACSEmployment


## 6) COMPAS

In [12]:
# PURPOSE: Load datasets and align schemas
# - Read ACSIncome, ACSEmployment (Folktables), COMPAS, CivilComments
# - Select features/target; cast types; attach sensitive attributes for EVAL ONLY


#@title COMPAS: Auto-download, preprocess, fairness experiments, plot + export
import os, pandas as pd, numpy as np
from urllib.request import urlretrieve
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds
from fairlearn.postprocessing import ThresholdOptimizer
from sklearn.tree import DecisionTreeClassifier

# ---------- 1) Download & Load ----------
compas_dir = "/content/data/compas"
os.makedirs(compas_dir, exist_ok=True)
compas_csv = os.path.join(compas_dir, "compas-scores-two-years.csv")

if not os.path.exists(compas_csv):
    print("Downloading COMPAS from ProPublica GitHub...")
    url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
    urlretrieve(url, compas_csv)

df = pd.read_csv(compas_csv)

# ---------- 2) Minimal, reproducible preprocessing ----------
# Keep common fields used in many public baselines
cols_keep = [
    "age", "juv_fel_count", "juv_misd_count", "juv_other_count",
    "priors_count", "c_charge_degree", "race", "sex", "two_year_recid"
]
df = df[cols_keep].dropna()

# Label: two_year_recid (1=reoffended within two years)
yC = df["two_year_recid"].astype(int).values

# Sensitive attribute: race (African-American=1, others=0)
gC = (df["race"].astype(str).str.strip() == "African-American").astype(int).values

# Features: numeric + one-hot for c_charge_degree and sex
X_num = df[["age", "juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count"]].astype(float)
X_cat = pd.get_dummies(df[["c_charge_degree", "sex"]].astype(str), drop_first=True)
XC = pd.concat([X_num, X_cat], axis=1).values

# ---------- 3) Train/test split + scaling (uses your split_scale helper) ----------
XtrC, XteC, ytrC, yteC, gtrC, gteC, scC = split_scale(XC, yC, gC)

# ---------- 4) Baseline ----------
baseC = LogisticRegression(max_iter=200, n_jobs=-1)
baseC.fit(XtrC, ytrC)
ysCb = baseC.predict_proba(XteC)[:, 1]
ypCb = (ysCb >= 0.5).astype(int)
mCb = evaluate_binary(yteC, ysCb, ypCb, gteC)
add_summary_row("compas", "baseline", mCb)
print("✅ COMPAS Baseline"); display(mCb)

# ---------- 5) DSAP-style reweighting (pre) ----------
wC = compute_sample_weight("balanced", ytrC)
lr_dsapC = LogisticRegression(max_iter=200, n_jobs=-1)
lr_dsapC.fit(XtrC, ytrC, sample_weight=wC)
ysCd = lr_dsapC.predict_proba(XteC)[:, 1]
ypCd = (ysCd >= 0.5).astype(int)
mCd = evaluate_binary(yteC, ysCd, ypCd, gteC)
add_summary_row("compas", "dsap", mCd)
print("✅ COMPAS DSAP"); display(mCd)

# ---------- 6) Equalized Odds via ExponentiatedGradient (in) ----------
egC = ExponentiatedGradient(
    DecisionTreeClassifier(max_depth=4, random_state=0),
    EqualizedOdds(),
    eps=0.02
)
egC.fit(XtrC, ytrC, sensitive_features=gtrC)
ysCe = eg_probabilities(egC, XteC)          # uses your helper
ypCe = (ysCe >= 0.5).astype(int)
mCe = evaluate_binary(yteC, ysCe, ypCe, gteC)
add_summary_row("compas", "eq_odds_eg", mCe)
print("✅ COMPAS Equalized Odds (EG)"); display(mCe)

# ---------- 7) Threshold Optimizer on Calibrated Scores (post) ----------
# Split a small calibration/validation fold from the training split
Xc_trC, Xc_valC, yc_trC, yc_valC, gc_trC, gc_valC = train_test_split(
    XtrC, ytrC, gtrC, test_size=0.2, random_state=3, stratify=ytrC
)

# Base classifier for calibration
base_lrC = LogisticRegression(max_iter=200, n_jobs=-1)
base_lrC.fit(Xc_trC, yc_trC)

# Calibrator (cv=3 -> no prefit headaches)
from sklearn.calibration import CalibratedClassifierCV
calC = CalibratedClassifierCV(base_lrC, cv=3, method="isotonic")
calC.fit(Xc_trC, yc_trC)

# Threshold Optimizer for Equalized Odds
toC = ThresholdOptimizer(
    estimator=calC,
    constraints="equalized_odds",
    predict_method="predict_proba"
)
toC.fit(Xc_valC, yc_valC, sensitive_features=gc_valC)

# Evaluate using your helper (calibrated probs + post-processed labels)
mC_to = evaluate_threshold_optimizer(calC, toC, XteC, yteC, gteC, "compas")
print("✅ COMPAS Threshold Optimizer"); display(mC_to)

# ---------- 8) Trade-off plot + CSV export ----------
_ = export_tradeoff_points(
    [
        {"label":"baseline",   "accuracy": mCb["accuracy"],  "eo_tpr_gap": mCb["eo_tpr_gap"]},
        {"label":"dsap",       "accuracy": mCd["accuracy"],  "eo_tpr_gap": mCd["eo_tpr_gap"]},
        {"label":"eq_odds",    "accuracy": mCe["accuracy"],  "eo_tpr_gap": mCe["eo_tpr_gap"]},
        {"label":"thresh_opt", "accuracy": mC_to["accuracy"],"eo_tpr_gap": mC_to["eo_tpr_gap"]},
    ],
    dataset_key="compas",
    title="COMPAS: Utility vs Fairness (EO gap)"
)

print("✅ Exported: /content/exports/compas_points.csv and /content/exports/compas_tradeoff.png")

✅ COMPAS Baseline


{'accuracy': 0.6757206208425721,
 'auc': 0.7235370486903658,
 'eo_tpr_gap': 0.27328938056393615,
 'eqodds_tpr_gap': 0.27328938056393615,
 'eqodds_tnr_gap': 0.15988265043085426}

✅ COMPAS DSAP


{'accuracy': 0.6729490022172949,
 'auc': 0.7236214491307376,
 'eo_tpr_gap': 0.29088911237754655,
 'eqodds_tpr_gap': 0.29088911237754655,
 'eqodds_tnr_gap': 0.2068511198945976}

✅ COMPAS Equalized Odds (EG)


{'accuracy': 0.5842572062084257,
 'auc': 0.6362849904987445,
 'eo_tpr_gap': 0.08850150854844106,
 'eqodds_tpr_gap': 0.08850150854844106,
 'eqodds_tnr_gap': 0.002798713573760825}

✅ COMPAS Threshold Optimizer


{'accuracy': 0.656319290465632,
 'auc': 0.7227681358549206,
 'eo_tpr_gap': 0.03246172756732624,
 'eqodds_tpr_gap': 0.03246172756732624,
 'eqodds_tnr_gap': 0.01738557598671009}

✅ Exported: /content/exports/compas_points.csv and /content/exports/compas_tradeoff.png


In [13]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output

#@title Methods + plot (COMPAS) — fixed & consistent
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds
from fairlearn.postprocessing import ThresholdOptimizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
import numpy as np
import pandas as pd

# ---------- Baseline ----------
bC = LogisticRegression(max_iter=400, n_jobs=-1).fit(XtrC, ytrC)
ysC = bC.predict_proba(XteC)[:, 1]
ypC = (ysC >= 0.5).astype(int)
mCb = evaluate_binary(yteC, ysC, ypC, gteC)
add_summary_row("compas", "baseline", mCb)

# ---------- DSAP-style reweighting (pre) ----------
wC = compute_sample_weight("balanced", ytrC)  # simple DSAP-style weight
dC = LogisticRegression(max_iter=400, n_jobs=-1).fit(XtrC, ytrC, sample_weight=wC)
ysCd = dC.predict_proba(XteC)[:, 1]
ypCd = (ysCd >= 0.5).astype(int)
mCd = evaluate_binary(yteC, ysCd, ypCd, gteC)
add_summary_row("compas", "dsap", mCd)

# ---------- Equalized Odds (ExponentiatedGradient) ----------
egC = ExponentiatedGradient(
    DecisionTreeClassifier(max_depth=4, random_state=0),
    EqualizedOdds(),
    eps=0.02
)
egC.fit(XtrC, ytrC, sensitive_features=gtrC)
ysCe = eg_probabilities(egC, XteC)      # helper you added
ypCe = (ysCe >= 0.5).astype(int)
mCe = evaluate_binary(yteC, ysCe, ypCe, gteC)
add_summary_row("compas", "eq_odds_eg", mCe)

# ---------- Threshold Optimizer on calibrated scores (post) ----------
# Split off validation from train for threshold optimization
Xc_trC, Xc_valC, yc_trC, yc_valC, gc_trC, gc_valC = train_test_split(
    XtrC, ytrC, gtrC, test_size=0.2, random_state=3, stratify=ytrC
)

# Fit base model for calibration
base_lrC = LogisticRegression(max_iter=400, n_jobs=-1)
base_lrC.fit(Xc_trC, yc_trC)

# Calibrate probabilities (cv=3 avoids prefit errors)
calC = CalibratedClassifierCV(base_lrC, cv=3, method="isotonic")
calC.fit(Xc_trC, yc_trC)

# Fairlearn Threshold Optimizer (Equalized Odds)
toC = ThresholdOptimizer(
    estimator=calC,
    constraints="equalized_odds",
    predict_method="predict_proba"
)
toC.fit(Xc_valC, yc_valC, sensitive_features=gc_valC)

# Evaluate via helper: calibrated probs + EO labels
mCt = evaluate_threshold_optimizer(calC, toC, XteC, yteC, gteC, "compas")

# ---------- Trade-off plot + CSV export (standardized filenames) ----------
_ = export_tradeoff_points(
    [
        {"label":"baseline",   "accuracy": mCb["accuracy"], "eo_tpr_gap": mCb["eo_tpr_gap"]},
        {"label":"dsap",       "accuracy": mCd["accuracy"], "eo_tpr_gap": mCd["eo_tpr_gap"]},
        {"label":"eq_odds",    "accuracy": mCe["accuracy"], "eo_tpr_gap": mCe["eo_tpr_gap"]},
        {"label":"thresh_opt", "accuracy": mCt["accuracy"], "eo_tpr_gap": mCt["eo_tpr_gap"]},
    ],
    dataset_key="compas",
    title="COMPAS: Utility vs Fairness (EO gap)"
)

## 7) CivilComments — manual upload + baseline

Steps: Upload `train.csv` → move to `/content/data/civilcomments/train.csv` → verify → run model.

In [15]:

#@title Move uploaded train.csv
import os, shutil
src="/content/train.csv"; dst_dir="/content/data/civilcomments"; dst=f"{dst_dir}/train.csv"
os.makedirs(dst_dir, exist_ok=True)
if os.path.exists(src): shutil.move(src, dst)
print("Exists:", os.path.exists(dst), dst)

Exists: True /content/data/civilcomments/train.csv


In [16]:

#@title Verify
!ls -l /content/data/civilcomments || echo "Missing directory"

total 797088
-rw-r--r-- 1 root root 816211476 Nov  4 02:50 train.csv


In [18]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output

#@title Baseline DistilBERT + trade-off plot (W&B disabled, Trainer.predict eval)

# 🚫 Disable Weights & Biases before importing transformers (belt & suspenders)
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"
os.environ["DISABLE_WANDB"] = "true"

import pandas as pd, numpy as np, torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --- Load data ---
p = "/content/data/civilcomments/train.csv"
if not os.path.exists(p):
    raise FileNotFoundError("Upload train.csv first (see instructions cell).")

df = pd.read_csv(p)
text_col = "comment_text" if "comment_text" in df.columns else "text"
label_col = "target" if "target" in df.columns else ("toxicity" if "toxicity" in df.columns else None)
assert text_col in df.columns and label_col in df.columns, "Need text/comment_text and target/toxicity columns."

# Binary label: >= 0.5 -> toxic
df = df[[text_col, label_col]].rename(columns={text_col: "text", label_col: "label"}).dropna()
df["label"] = (df["label"] >= 0.5).astype(int)

# Small sample for speed
df = df.sample(n=min(12000, len(df)), random_state=42)

tr, te = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

# --- Tokenization ---
tok = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokf(batch):
    return tok(batch["text"], truncation=True, padding="max_length", max_length=128)

ds_tr = Dataset.from_pandas(tr).map(tokf, batched=True).rename_column("label", "labels")
ds_te = Dataset.from_pandas(te).map(tokf, batched=True).rename_column("label", "labels")
ds_tr.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
ds_te.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# --- Model ---
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Trainer metrics helper (expects numpy arrays)
def cm(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):  # some HF versions
        logits = logits[0]
    preds = (logits[:, 1] >= 0.5).astype(int)
    return {"accuracy": accuracy_score(labels, preds)}

# --- TrainingArguments: no reporting/logging that triggers W&B ---
args = TrainingArguments(
    output_dir="/content/cc_out",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=3e-5,
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_strategy="no",
    report_to="none",      # <- critical: disables wandb/tensorboard/mlflow
    run_name=None,
    load_best_model_at_end=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tr,
    eval_dataset=ds_te,
    tokenizer=tok,
    compute_metrics=cm,
)

# --- Train ---
trainer.train()

# --- Evaluate using Trainer.predict (handles device automatically) ---
pred = trainer.predict(ds_te)
logits = pred.predictions if not isinstance(pred.predictions, tuple) else pred.predictions[0]
labels = pred.label_ids

probs = torch.softmax(torch.from_numpy(logits), dim=-1)[:, 1].numpy()
y_score = probs
y_pred  = (y_score >= 0.5).astype(int)
g_test  = np.zeros(len(y_pred), dtype=int)   # no group labels available for CivilComments baseline

# --- Metrics, export, plot ---
m_cc = evaluate_binary(labels, y_score, y_pred, g_test)
add_summary_row("civilcomments", "baseline_distilbert", m_cc)

dfp = pd.DataFrame([{"label": "baseline", "accuracy": m_cc["accuracy"], "eo_tpr_gap": m_cc["eo_tpr_gap"]}])
png = "/content/exports/civilcomments_tradeoff.png"
tradeoff_plot(dfp, "accuracy", "eo_tpr_gap", "CivilComments: Utility vs Fairness (EO gap)", png)
save_results_csv(dfp, "/content/exports/civilcomments_points.csv")
flush_summary_csv()
png


Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipython-input-800341697.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.175457,0.939167


'/content/exports/civilcomments_tradeoff.png'

## 8) Vision (optional) — DeAR (toy) + FRAME/SLSD stubs

## 8) Exports (Figures + CSV + Overleaf)


In [19]:
# PURPOSE: Export artifacts for paper and slides
# - Save tradeoff plots (income/employment/compas/civilcomments)
# - Save consolidated results_summary_all.csv
# - (Optional) Package Overleaf zip


#@title FairFace-like tiny subset (placeholders)
import os, pandas as pd, numpy as np
from PIL import Image
FAIR_DIR="/content/data/fairface"; os.makedirs(FAIR_DIR, exist_ok=True)
meta=f"{FAIR_DIR}/subset.csv"
if not os.path.exists(meta):
    rows=[]
    for i in range(60):
        img=Image.new("RGB",(64,64),(50+3*i%255,100+5*i%255,150+7*i%255))
        p=f"{FAIR_DIR}/img_{i}.png"; img.save(p); rows.append({"img_path":p})
    pd.DataFrame(rows).to_csv(meta, index=False)
print("Subset ready:", os.path.exists(meta))

Subset ready: True


In [20]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output


#@title DeAR residualization on embeddings (toy)
import numpy as np, pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

meta="/content/data/fairface/subset.csv"
if not os.path.exists(meta):
    print("Skipping DeAR demo.")
else:
    df=pd.read_csv(meta)
    Z=[]
    for p in df["img_path"]:
        arr=np.array(Image.open(p)).astype(np.float32)/255.0
        Z.append([arr.mean(), arr.std(), arr[...,0].mean(), arr[...,1].mean(), arr[...,2].mean()])
    Z=np.asarray(Z); y=(Z[:,0]>np.median(Z[:,0])).astype(int); g=(Z[:,3]>np.median(Z[:,3])).astype(int)
    Xtr,Xte,ytr,yte,gtr,gte=train_test_split(Z,y,g,test_size=0.3,random_state=1,stratify=y)
    sc=StandardScaler(); Xtr=sc.fit_transform(Xtr); Xte=sc.transform(Xte)
    base=LogisticRegression().fit(Xtr,ytr); ys=base.predict_proba(Xte)[:,1]; yp=(ys>=0.5).astype(int)
    m_b=evaluate_binary(yte, ys, yp, gte); add_summary_row("fairface","baseline_toy",m_b)
    def residualize(Z,g):
        Zr=Z.copy().astype(float); A=g.reshape(-1,1); ATA=np.linalg.pinv(A.T@A + 1e-6)
        for j in range(Z.shape[1]):
            z=Z[:,j:j+1]; beta=ATA @ (A.T@z); Zr[:,j]=(z - A@beta).ravel()
        return Zr
    Zr=residualize(Z,g); Xtr2,Xte2,ytr2,yte2,gtr2,gte2=train_test_split(Zr,y,g,test_size=0.3,random_state=1,stratify=y)
    Xtr2=sc.fit_transform(Xtr2); Xte2=sc.transform(Xte2)
    dear=LogisticRegression().fit(Xtr2,ytr2); ys2=dear.predict_proba(Xte2)[:,1]; yp2=(ys2>=0.5).astype(int)
    m_d=evaluate_binary(yte2, ys2, yp2, gte2); add_summary_row("fairface","dear_residual",m_d)
    import pandas as pd
    dfv=pd.DataFrame([{"label":"baseline","accuracy":m_b["accuracy"],"eo_tpr_gap":m_b["eo_tpr_gap"]},
                      {"label":"dear","accuracy":m_d["accuracy"],"eo_tpr_gap":m_d["eo_tpr_gap"]}])
    png="/content/exports/fairface_dear_tradeoff.png"; tradeoff_plot(dfv,"accuracy","eo_tpr_gap","FairFace (toy): Utility vs Fairness",png)
    save_results_csv(dfv,"/content/exports/fairface_points.csv"); flush_summary_csv(); png

In [21]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output


#@title FRAME & SLSD stubs
FRAME_DESCRIPTION="FRAME: augmentation/ensemble-aware reweighting placeholder."
SLSD_DESCRIPTION="SLSD: latent transform neutralizing single sensitive direction (placeholder)."
def FRAME_stub_sample_weights(X,y,g,policy="uniform"):
    import numpy as np; return np.ones(len(y),dtype=float)
def SLSD_stub_transform(Z,g): return Z
print("FRAME/SLSD stubs loaded.")

FRAME/SLSD stubs loaded.


## 9) Aggregated Results

In [22]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output


#@title Write CSV & preview
flush_summary_csv("/content/exports/results_summary_all.csv")
import pandas as pd; pd.read_csv("/content/exports/results_summary_all.csv").head()

,dataset,method,accuracy,auc,eo_tpr_gap,eqodds_tpr_gap,eqodds_tnr_gap
0,acs_income,baseline,0.784574,0.859782,0.092418,0.092418,0.041504
1,acs_income,dsap_reweight,0.778584,0.858756,0.018239,0.018239,0.000313
2,acs_income,eq_odds_eg,0.773555,0.784357,0.008966,0.008966,0.009792
3,acs_income,threshold_optimizer,0.779381,0.859844,0.013763,0.013763,0.000153
4,acs_employment,baseline,0.771607,0.845225,0.017761,0.017761,0.026776


## 10) Export & Publish

In [25]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output

#@title Sanitize, release bundle, Overleaf pack
import os, glob, shutil, zipfile, nbformat, pandas as pd

SANITIZED="/content/AIM460_Fairness_Benchmark_Final_clean.ipynb"
RELEASE="/content/release_bundle"; OVERLEAF="/content/overleaf_pack"; EXPORTS="/content/exports"
os.makedirs(RELEASE, exist_ok=True); os.makedirs(OVERLEAF, exist_ok=True); os.makedirs(EXPORTS, exist_ok=True)

# Sanitize: remove widgets metadata from the most recent .ipynb in CWD
cands=sorted(glob.glob("*.ipynb"), key=os.path.getmtime, reverse=True)
if cands:
    nb=nbformat.read(cands[0], as_version=4); nb.metadata.pop("widgets", None)
    with open(SANITIZED,"w") as f: nbformat.write(nb,f); print("Sanitized:", SANITIZED)
else:
    print("Save notebook first so it appears in CWD.")

# Copy exports + sanitized into release
if os.path.exists(SANITIZED): shutil.copy(SANITIZED, RELEASE)
for p in glob.glob(f"{EXPORTS}/*"): shutil.copy(p, RELEASE)

# Slides (pptx) placeholders
from pptx import Presentation
prs=Presentation()
s=prs.slides.add_slide(prs.slide_layouts[0]); s.shapes.title.text="Cross-Domain Algorithmic Fairness Benchmark"; s.placeholders[1].text="AIM460 Capstone"
def slide(title, bullets, notes=None):
    sl=prs.slides.add_slide(prs.slide_layouts[1]); sl.shapes.title.text=title; tf=sl.placeholders[1].text_frame; tf.clear()
    for i,b in enumerate(bullets):
        p=tf.add_paragraph() if i else tf.paragraphs[0]; p.text=b
    if notes: sl.notes_slide.notes_text_frame.text=notes
slide("Methods", ["DSAP (pre)", "Eq-Odds (EG)", "Threshold Optimizer", "FRAME/SLSD stubs"], "Axes: Accuracy ↑; EO gap ↓")
slide("Datasets", ["ACSIncome", "ACSEmployment", "COMPAS", "CivilComments", "FairFace (opt)"])
slide("Tabular Results", ["income_tradeoff.png", "employment_tradeoff.png"])
slide("COMPAS", ["compas_tradeoff.png"])
slide("Text & Vision", ["civilcomments_tradeoff.png", "fairface_dear_tradeoff.png"])
slide("Key Takeaways", ["No single method dominates", "Context matters", "Calibration helps post-processing"])
slides_path=os.path.join(RELEASE,"AIM460_slides.pptx"); prs.save(slides_path)

# Overleaf: write files without raw string issues
main_lines=[
    r"\documentclass[conference]{IEEEtran}",
    r"\usepackage{graphicx}",
    r"\usepackage{booktabs}",
    r"\usepackage{hyperref}",
    r"\begin{document}",
    r"\title{Cross-Domain Algorithmic Fairness Benchmark}",
    r"\author{AIM460 Capstone}",
    r"\maketitle",
    r"\begin{abstract}",
    r"We present a cross-domain benchmark evaluating pre-, in-, and post-processing fairness methods across multiple modalities.",
    r"\end{abstract}",
    r"\section{Introduction}",
    r"% \cite{intro}",
    r"\section{Related Work}",
    r"% \cite{fairlearn,frame,dsap,slsd,dear}",
    r"\section{Methods}",
    r"Pre: DSAP. In: Equalized Odds (Exponentiated Gradient). Post: Threshold Optimizer (calibrated). FRAME/SLSD described and stubbed.",
    r"\section{Datasets}",
    r"ACSIncome, ACSEmployment, COMPAS, CivilComments, FairFace (toy).",
    r"\section{Experimental Setup}",
    r"Splits, calibration, sensitive groups.",
    r"\section{Results}",
    r"See Figs.~\ref{fig:inc}, \ref{fig:emp}, \ref{fig:compas}, \ref{fig:civil}, \ref{fig:fairface}.",
    r"\begin{figure}[t]\centering\includegraphics[width=0.9\linewidth]{income_tradeoff.png}\caption{ACSIncome: Accuracy (\uparrow) vs EO gap (\downarrow).}\label{fig:inc}\end{figure}",
    r"\begin{figure}[t]\centering\includegraphics[width=0.9\linewidth]{employment_tradeoff.png}\caption{ACSEmployment: Accuracy (\uparrow) vs EO gap (\downarrow).}\label{fig:emp}\end{figure}",
    r"\begin{figure}[t]\centering\includegraphics[width=0.9\linewidth]{compas_tradeoff.png}\caption{COMPAS: Accuracy (\uparrow) vs EO gap (\downarrow).}\label{fig:compas}\end{figure}",
    r"\begin{figure}[t]\centering\includegraphics[width=0.9\linewidth]{civilcomments_tradeoff.png}\caption{CivilComments: Accuracy (\uparrow) vs EO gap (\downarrow).}\label{fig:civil}\end{figure}",
    r"\begin{figure}[t]\centering\includegraphics[width=0.9\linewidth]{fairface_dear_tradeoff.png}\caption{FairFace: Accuracy (\uparrow) vs EO gap (\downarrow).}\label{fig:fairface}\end{figure}",
    r"\input{results_summary_all.tex}",
    r"\section{Discussion}",
    r"Utility/fairness trade-offs across domains.",
    r"\section{Limitations}",
    r"Proxies for sensitive groups; small subsamples for speed.",
    r"\section{Conclusion}",
    r"Summary and future work.",
    r"\bibliographystyle{IEEEtran}",
    r"\bibliography{references}",
    r"\end{document}",
]
os.makedirs(OVERLEAF, exist_ok=True)
with open(os.path.join(OVERLEAF,"main.tex"),"w") as f: f.write("\n".join(main_lines))

csv="/content/exports/results_summary_all.csv"
with open(os.path.join(OVERLEAF,"results_summary_all.tex"),"w") as f:
    if os.path.exists(csv):
        import pandas as pd; df=pd.read_csv(csv); f.write(df.to_latex(index=False, float_format="%.3f"))
    else:
        f.write("% Generate results CSV first.\n")

with open(os.path.join(OVERLEAF,"references.bib"),"w") as f: f.write("% Add your BibTeX entries here.\n")

# Copy exports into Overleaf
for p in glob.glob(f"{EXPORTS}/*"): shutil.copy(p, OVERLEAF)

# ZIPs
def make_zip(root, out):
    with zipfile.ZipFile(out, "w", zipfile.ZIP_DEFLATED) as z:
        for folder,_,files in os.walk(root):
            for fn in files:
                fp=os.path.join(folder, fn); z.write(fp, os.path.relpath(fp, root))

release_zip="/content/AIM460_release_bundle.zip"; overleaf_zip="/content/overleaf_pack.zip"
make_zip(RELEASE, release_zip); make_zip(OVERLEAF, overleaf_zip)
print("Created:", release_zip); print("Created:", overleaf_zip)

Save notebook first so it appears in CWD.
Created: /content/AIM460_release_bundle.zip
Created: /content/overleaf_pack.zip


In [29]:
# === DIAGNOSE + FIX NOTEBOOK PATHS (pick by index, auto-rename to clean paths) ===
import os, glob, shutil
from pathlib import Path

# 1) Find all ipynb files under /content (depth 3)
candidates = []
for pattern in ["/*.ipynb","/*/*.ipynb","/*/*/*.ipynb"]:
    for p in glob.glob("/content" + pattern):
        candidates.append(Path(p))
candidates = sorted(set(candidates))

print("Found these .ipynb files under /content:\n")
for i,p in enumerate(candidates):
    print(f"[{i}] {p}")

if not candidates:
    raise SystemExit("No .ipynb files found under /content. If your files are in Drive, mount and copy them:\n"
                     "from google.colab import drive; drive.mount('/content/drive')\n"
                     "!cp '/content/drive/MyDrive/your/folder/*.ipynb' /content/")

# 2) >>> EDIT THESE TWO INDICES to select your NEWER and OLDER notebooks <<<
NEW_IDX = 0   # <-- put the index of Maurrasse_AIM460_Fairness_Benchmark_(1).ipynb
OLD_IDX = 1   # <-- put the index of aim460_fairness_project_complete_final_aligned (2).ipynb

assert 0 <= NEW_IDX < len(candidates), "NEW_IDX out of range"
assert 0 <= OLD_IDX < len(candidates), "OLD_IDX out of range"
NEW_SRC = candidates[NEW_IDX]
OLD_SRC = candidates[OLD_IDX]

# 3) Copy to clean, canonical names (avoid spaces/parentheses)
NEW_DST = Path("/content/newer.ipynb")
OLD_DST = Path("/content/older.ipynb")
if NEW_DST.exists(): NEW_DST.unlink()
if OLD_DST.exists(): OLD_DST.unlink()
shutil.copy2(NEW_SRC, NEW_DST)
shutil.copy2(OLD_SRC, OLD_DST)

print("\n✅ Canonical paths set:")
print("NEW_NB =", NEW_DST)
print("OLD_NB =", OLD_DST)

# 4) Sanity check that /content/exports exists
exp = Path("/content/exports")
print("\nExports folder:", exp, "exists?" , exp.exists())
if not exp.exists():
    print("⚠️ /content/exports is missing. Re-run your newer notebook to generate it.")


Found these .ipynb files under /content:

[0] /content/Maurrasse_AIM460_Fairness_Benchmark (1).ipynb
[1] /content/aim460_fairness_project_complete_final_aligned (2).ipynb

✅ Canonical paths set:
NEW_NB = /content/newer.ipynb
OLD_NB = /content/older.ipynb

Exports folder: /content/exports exists? True


In [32]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output

# === ONE-CELL FINALIZER ===
# Assumes /content/exports/ already exists with PNGs + results_summary_all.csv

# ---- EDIT THESE 2 PATHS IF NEEDED ----
NEW_NB = "/content/newer.ipynb"  # newer, correct
OLD_NB = "/content/older.ipynb"  # older, better documentation


# --------------------------------------

import os, json, re, shutil, subprocess, sys
from pathlib import Path
import pandas as pd, numpy as np

root = Path("/content")
exports = root / "exports"
assert Path(NEW_NB).exists(), f"Missing newer notebook at {NEW_NB}"
assert Path(OLD_NB).exists(), f"Missing older notebook at {OLD_NB} (upload it to /content)"
assert exports.exists(), "Missing /content/exports (expected plots + CSVs)"

# ---------- 1) Release bundle ----------
bundle_dir = root / "AIM460_release_bundle"
bundle_zip = root / "AIM460_release_bundle.zip"
if bundle_dir.exists(): shutil.rmtree(bundle_dir)
bundle_dir.mkdir(parents=True)

# copy newer notebook + exports
shutil.copy2(NEW_NB, bundle_dir / Path(NEW_NB).name)
shutil.copytree(exports, bundle_dir / "exports")

# environment freeze + requirements
env_file = bundle_dir / "environment_freeze.txt"
try:
    reqs = subprocess.check_output([sys.executable, "-m", "pip", "freeze"], text=True)
    env_file.write_text(reqs)
except Exception as e:
    env_file.write_text(f"Could not pip freeze: {e}\n")

reqs_path = bundle_dir / "requirements.txt"
lines = env_file.read_text().splitlines()
keep_prefixes = ("numpy","pandas","scikit-learn","matplotlib","torch","transformers","datasets","tqdm","fairlearn")
picked = [ln for ln in lines if ln.lower().startswith(keep_prefixes)]
reqs_path.write_text("\n".join(picked or lines))

(bundle_dir / "README.md").write_text(
"""# AIM460 Cross-Domain Algorithmic Fairness Benchmark
Contents:
- Final notebook
- exports/ (tradeoff PNGs + CSV summaries)
- environment_freeze.txt, requirements.txt

Re-run: `pip install -r requirements.txt`, open notebook, Run All.
"""
)

if bundle_zip.exists(): bundle_zip.unlink()
shutil.make_archive(str(bundle_zip.with_suffix("")), "zip", bundle_dir)

# ---------- 2) Overleaf pack ----------
overleaf_dir = root / "overleaf_pack"
overleaf_zip = root / "overleaf_pack.zip"
if overleaf_dir.exists(): shutil.rmtree(overleaf_dir)
figdir = overleaf_dir / "figures"
figdir.mkdir(parents=True)

# copy canonical figures + any *tradeoff*.png
wanted = ["income_tradeoff.png","employment_tradeoff.png","compas_tradeoff.png","civilcomments_tradeoff.png"]
present = set()
for name in wanted:
    p = exports / name
    if p.exists(): shutil.copy2(p, figdir / name); present.add(name)
for p in exports.glob("*tradeoff*.png"):
    dst = figdir / p.name
    if not dst.exists(): shutil.copy2(p, dst); present.add(p.name)

# copy results summary csv if present
summary_csv = exports / "results_summary_all.csv"
has_csv = summary_csv.exists()
if has_csv:
    shutil.copy2(summary_csv, overleaf_dir / "results_summary_all.csv")

# skeleton main.tex with placeholders
main_tex = r"""
\documentclass[conference]{IEEEtran}
\IEEEoverridecommandlockouts
\usepackage{graphicx}
\usepackage{booktabs}
\usepackage{amsmath}
\usepackage{hyperref}
\usepackage{siunitx}
\usepackage{xcolor}

\title{Cross-Domain Algorithmic Fairness Benchmark}
\author{\IEEEauthorblockN{Your Name}
\IEEEauthorblockA{AIM460 Capstone \\\texttt{youremail@example.com}}}
\begin{document}\maketitle

\begin{abstract}
We present a reproducible benchmark for algorithmic fairness across multiple datasets and modalities.
\end{abstract}

\section{Introduction}
Brief overview...

\section{Related Work}
Foundational literature...

\section{Methods}
% <<< INJECT: METHODS_FROM_OLD_NOTEBOOK >>>

\section{Datasets}
We include COMPAS, CivilComments, and Folktables tasks (income/employment)...

\section{Experimental Setup}
Unified splits; Fairlearn metrics; mitigation strategies...

\section{Results}
% <<< INJECT: RESULTS_TABLES >>>

\begin{figure}[!t]\centering
\includegraphics[width=\linewidth]{figures/income_tradeoff.png}
\caption{Income: accuracy--fairness trade-off.}
\end{figure}

\begin{figure}[!t]\centering
\includegraphics[width=\linewidth]{figures/employment_tradeoff.png}
\caption{Employment: accuracy--fairness trade-off.}
\end{figure}

\begin{figure}[!t]\centering
\includegraphics[width=\linewidth]{figures/compas_tradeoff.png}
\caption{COMPAS: accuracy--fairness trade-off.}
\end{figure}

\begin{figure}[!t]\centering
\includegraphics[width=\linewidth]{figures/civilcomments_tradeoff.png}
\caption{CivilComments: accuracy--fairness trade-off.}
\end{figure}

\section{Discussion}
Observed tensions and generalization notes...

\section{Limitations}
% <<< INJECT: LIMITATIONS_FROM_OLD_NOTEBOOK >>>

\section{Conclusion}
Release bundle and teaching utility…

\bibliographystyle{IEEEtran}
\bibliography{references}
\end{document}
""".strip()
(overleaf_dir / "main.tex").write_text(main_tex)

(overleaf_dir / "references.bib").write_text(r"""
@inproceedings{hardt2016equality,
  title={Equality of Opportunity in Supervised Learning},
  author={Hardt, Moritz and Price, Eric and Srebro, Nati},
  booktitle={NeurIPS}, year={2016}
}
@book{barocas-hardt-narayanan,
  title={Fairness and Machine Learning}, author={Barocas, Hardt, Narayanan}, year={2019},
  note={\url{https://fairmlbook.org}}
}
@article{mehrabi2019survey,
  title={A Survey on Bias and Fairness in Machine Learning},
  author={Mehrabi et al.}, journal={arXiv:1908.09635}, year={2019}
}
""".strip())

# ---------- 3) Lift Methods & Limitations from OLD notebook ----------
def load_md_cells(ipynb_path):
    obj = json.loads(Path(ipynb_path).read_text())
    return ["".join(c.get("source","")) for c in obj.get("cells",[]) if c.get("cell_type")=="markdown"]

md_cells = load_md_cells(OLD_NB)

method_keys = [
    "pre-processing","post-processing","in-processing",
    "Equalized Odds","Equalized Opportunity","ThresholdOptimizer",
    "reweigh","reweight","DSAP","reduction","mitigation"
]
limit_keys = ["limitation","caveat","bias","label noise","selection bias","calibration"]

def pick_blocks(cells, keys, k=6):
    out, seen = [], set()
    for c in cells:
        if any(k_.lower() in c.lower() for k_ in keys):
            c = c.strip()
            if c and c not in seen:
                out.append(c); seen.add(c)
    return out[:k]

methods_blocks = pick_blocks(md_cells, method_keys)
limits_blocks = pick_blocks(md_cells, limit_keys)

import re
def md_to_tex(md):
    md = md.replace("**", r"\textbf{").replace("__", r"\textbf{")
    md = re.sub(r"\*(.+?)\*", r"\\emph{\1}", md)
    md = re.sub(r"^[-*]\s+", r"\\item ", md, flags=re.M)
    if "\\item " in md and "\\begin{itemize}" not in md:
        md = "\\begin{itemize}\n" + md + "\n\\end{itemize}"
    return md

methods_tex = "\n\n".join(md_to_tex(b) for b in methods_blocks) or \
              "We apply pre-, in-, and post-processing mitigations including reweighting, reduction-based Equalized Odds, and threshold optimization."
limits_tex  = "\n\n".join(md_to_tex(b) for b in limits_blocks) or \
              "We focus on group metrics; dataset artifacts (e.g., label noise, selection bias) and calibration are not fully addressed."

mpath = overleaf_dir / "main.tex"
m = mpath.read_text()
m = m.replace("% <<< INJECT: METHODS_FROM_OLD_NOTEBOOK >>>", methods_tex)
m = m.replace("% <<< INJECT: LIMITATIONS_FROM_OLD_NOTEBOOK >>>", limits_tex)
mpath.write_text(m)

# also save notes file
docs_dir = root / "docs"; docs_dir.mkdir(exist_ok=True)
(docs_dir / "methods_notes.md").write_text("# Methods Notes (lifted from older notebook)\n\n" + "\n\n---\n\n".join(methods_blocks or ["(No extracted blocks—placeholder)"]))

# ---------- 4) Auto-build LaTeX results tables from existing CSV ----------
if has_csv:
    df = pd.read_csv(summary_csv)
    def norm(s): return s.strip().lower().replace(" ","_")
    df.columns = [norm(c) for c in df.columns]
    gcols = [c for c in ["dataset","task","domain","corpus","benchmark"] if c in df.columns]
    assert gcols, "Couldn't find a dataset-like column (need 'dataset' or 'task' in CSV)."
    gcol = gcols[0]
    mcols_all = [c for c in df.columns if c not in {gcol,"model","clf","estimator","mitigation","strategy","method","split","seed","run","notes"} and np.issubdtype(df[c].dtype, np.number)]
    preferred = ["accuracy","auroc","f1","demographic_parity_difference","equalized_odds_tpr_diff","equalized_odds_fpr_diff"]
    metrics = [m for m in preferred if m in mcols_all] or mcols_all
    pretty = {
        "accuracy":"Accuracy","auroc":"AUROC","f1":"F1","macro_f1":"Macro-F1",
        "demographic_parity_difference":"DPD (|Δ|)","equalized_odds_tpr_diff":"EO TPR Δ (|Δ|)","equalized_odds_fpr_diff":"EO FPR Δ (|Δ|)"
    }
    fairness_like = set(["demographic_parity_difference","equalized_odds_tpr_diff","equalized_odds_fpr_diff","calibration_error"])
    if "split" in df.columns:
        test_mask = df["split"].str.lower().eq("test")
        if test_mask.any(): df = df[test_mask].copy()
    if "model" not in df.columns: df["model"] = df.get("clf", df.get("estimator","(model)"))
    if "mitigation" not in df.columns: df["mitigation"] = df.get("strategy", df.get("method","(none)"))
    def fmt(x):
        if pd.isna(x): return ""
        return f"{x:.3f}" if abs(x) < 10 else f"{x:.2f}"
    def best_mask(s, fairness=False):
        if fairness: s_ = s.abs(); b = s_.min()
        else: b = s.max()
        return s.eq(b)
    tables=[]
    for ds, sub in df.groupby(gcol):
        cols = ["model","mitigation"] + [m for m in metrics if m in sub.columns]
        t = sub[cols].reset_index(drop=True).copy()
        # bold masks
        masks={}
        for mtr in metrics:
            if mtr in t.columns:
                masks[mtr]=best_mask(t[mtr], fairness=mtr in fairness_like)
        lines=[]
        for i,row in t.iterrows():
            cells=[str(row["model"]), str(row["mitigation"])]
            for mtr in metrics:
                if mtr not in t.columns: continue
                s=fmt(row[mtr])
                if masks[mtr].iloc[i]: s=f"\\textbf{{{s}}}"
                cells.append(s)
            lines.append(" & ".join(cells) + " \\\\")
        headers=["Model","Mitigation"] + [pretty.get(m, m.replace("_"," ").title()) for m in metrics if m in t.columns]
        colspec = "ll" + "c"*(len(headers)-2)
        tab = []
        tab += ["\\begin{table}[!t]","\\centering",
                f"\\caption{{{ds}: summary metrics and fairness deltas (best per column in bold).}}",
                f"\\label{{tab:{re.sub(r'[^a-zA-Z0-9]+','-', str(ds).lower())}}}",
                f"\\\\begin{{tabular}}{{{colspec}}}", "\\toprule",

                " & ".join(headers) + " \\\\","\\midrule"]
        tab += lines
        tab += ["\\bottomrule","\\end{tabular}","\\end{table}"]
        tables.append("\n\n".join(tab))
    (overleaf_dir / "results_tables.tex").write_text("\n\n".join(tables))
    # inject \input
    mt = (overleaf_dir / "main.tex").read_text()
    ph = "% <<< INJECT: RESULTS_TABLES >>>"
    snippet = "\n\n% Auto-generated results tables\n\\input{results_tables}\n"
    if ph in mt:
        mt = mt.replace(ph, snippet)
    else:
        mt = mt.replace("\\section{Discussion}", snippet + "\n\\section{Discussion}")
    (overleaf_dir / "main.tex").write_text(mt)

# zip Overleaf pack
if overleaf_zip.exists(): overleaf_zip.unlink()
shutil.make_archive(str(overleaf_zip.with_suffix("")), "zip", overleaf_dir)

print("✅ Done.")
print("Release bundle:", bundle_zip)
print("Overleaf pack:", overleaf_zip)
print("Figures included:", sorted(present))
print("Has results_summary_all.csv:", has_csv)

✅ Done.
Release bundle: /content/AIM460_release_bundle.zip
Overleaf pack: /content/overleaf_pack.zip
Figures included: ['civilcomments_tradeoff.png', 'compas_tradeoff.png', 'employment_tradeoff.png', 'fairface_dear_tradeoff.png', 'income_tradeoff.png']
Has results_summary_all.csv: True


In [34]:
# --- Fix GitHub render error: remove bad metadata.widgets and (optionally) clear outputs ---
import nbformat, shutil
from pathlib import Path

src = Path("/content/Maurrasse_AIM460_Fairness_Benchmark (1).ipynb")  # your current notebook
dst = Path("/content/Maurrasse_AIM460_Fairness_Benchmark_github.ipynb")  # clean copy for GitHub

nb = nbformat.read(src, as_version=4)

# If a widgets block exists but is malformed, drop it entirely (GitHub doesn’t need it to render).
widgets = nb.metadata.get("widgets", None)
if widgets is not None:
    # safest: remove whole block
    nb.metadata.pop("widgets", None)

# (Optional) also clear heavy outputs to keep file small and render fast.
for cell in nb.cells:
    if cell.get("cell_type") == "code":
        cell["outputs"] = []
        cell["execution_count"] = None

nbformat.write(nb, dst)
print("✅ Wrote cleaned notebook:", dst)

✅ Wrote cleaned notebook: /content/Maurrasse_AIM460_Fairness_Benchmark_github.ipynb


In [ ]:
# PURPOSE: Initialize environment and imports
# - Set random seeds (numpy/torch/sklearn)
# - Import pandas, numpy, matplotlib, sklearn, fairlearn, transformers
# - Configure warnings/logging for clean output

# === Cross-domain fairness–utility tradeoff summary plot ===
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Path to your consolidated results CSV (should already exist)
csv_path = "/content/exports/results_summary_all.csv"

df = pd.read_csv(csv_path)
assert {"dataset","method","accuracy","eqodds_tpr_gap"}.issubset(df.columns), "CSV missing key columns."

# Clean up dataset and method names for display
display_names = {
    "acs_income": "Income",
    "acs_employment": "Employment",
    "compas": "COMPAS",
    "civilcomments": "CivilComments",
    "fairface": "FairFace"
}
df["dataset_label"] = df["dataset"].map(display_names).fillna(df["dataset"])
df["method_label"] = df["method"].str.replace("_", " ").str.title()

# Build the plot
plt.figure(figsize=(7,5))
colors = {
    "Income":"#1f77b4",
    "Employment":"#2ca02c",
    "COMPAS":"#d62728",
    "CivilComments":"#9467bd",
    "FairFace":"#ff7f0e"
}

for dset, sub in df.groupby("dataset_label"):
    plt.scatter(
        sub["eqodds_tpr_gap"],
        sub["accuracy"],
        label=dset,
        s=80,
        alpha=0.8,
        color=colors.get(dset,"gray"),
        edgecolor="black"
    )
    # annotate best fairness per dataset
    best_idx = sub["eqodds_tpr_gap"].idxmin()
    if pd.notna(best_idx):
        x, y = sub.loc[best_idx, ["eqodds_tpr_gap","accuracy"]]
        plt.text(x, y+0.005, sub.loc[best_idx,"method_label"], fontsize=8, ha="center")

plt.xlabel("Equalized Odds TPR Gap (↓ Fairer)")
plt.ylabel("Accuracy (↑ Better)")
plt.title("Cross-Domain Fairness–Utility Trade-off")
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(title="Dataset", loc="best")
plt.tight_layout()

# Save the plot
outdir = Path("/content/exports"); outdir.mkdir(parents=True, exist_ok=True)
out_path = outdir / "all_datasets_tradeoff.png"
plt.savefig(out_path, dpi=300)
plt.close()
print(f"✅ Saved summary plot to {out_path}")


### ✅ Final Export Paths
- `/content/exports/income_tradeoff.png`
- `/content/exports/employment_tradeoff.png`
- `/content/exports/compas_tradeoff.png` (if run)
- `/content/exports/civilcomments_tradeoff.png` (if run)
- `/content/exports/fairface_dear_tradeoff.png` (if run)
- `/content/exports/results_summary_all.csv`
- Sanitized: `/content/AIM460_Fairness_Benchmark_Final_clean.ipynb`
- Release ZIP: `/content/AIM460_release_bundle.zip`
- Overleaf ZIP: `/content/overleaf_pack.zip`

## 3) Data Cleaning & Imputation (KNN etc.)

*Placeholder: add code or confirm this step was done earlier in the notebook.*


## 4) Feature Engineering

*Placeholder: add code or confirm this step was done earlier in the notebook.*


## 6) Metrics (Utility + Fairness)

*Placeholder: add code or confirm this step was done earlier in the notebook.*


## 7) Fairness Mitigations

*Placeholder: add code or confirm this step was done earlier in the notebook.*
